In [2]:
import json

In [3]:
with open("merged_dataset.json", "r") as file:
    dataset = json.loads(file.read())

In [4]:
print(len(dataset))

1114


In [5]:
from prompt_generator import CodeReviewPromptGenerator

In [6]:
prompt_generator_service = CodeReviewPromptGenerator()

In [7]:
prompt = prompt_generator_service.generate_style_review_prompt(
    added_code=dataset[0]['added_code'],
    deleted_code=dataset[0]['deleted_code'],
    full_function_code=dataset[0]['full_function_code'],
    function_name=dataset[0]['function_name']
)

In [8]:
print(prompt)

You are a code reviewer. Analyze this Python code change and respond EXACTLY in the format below.

Full function `handle_api_response`:
```python
def handle_api_response(api_response):
    if api_response is not None:
        if isinstance(api_response, dict):
            if "status_code" in api_response:
                status_code = api_response["status_code"]
                if status_code == 200:
                    response_data = api_response["data"]
                    # Validate response data
                    if isinstance(response_data, list):
                        for item in response_data:
                            if not isinstance(item, dict):
                                raise ValueError("Invalid response data")
                    elif not isinstance(response_data, dict):
                        raise ValueError("Invalid response data")
                    return response_data
                else:
                    raise ValueError("Invalid status code")
   

In [17]:
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("deepseek-ai/deepseek-coder-1.3b-instruct")
model = AutoModelForCausalLM.from_pretrained("deepseek-ai/deepseek-coder-1.3b-instruct")
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

In [10]:
# Format with chat template
messages = [
    {"role": "user", "content": prompt}
]
input_text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

In [11]:
# Generate
inputs = tokenizer(input_text, return_tensors="pt")
input_length = inputs.input_ids.shape[1]  # Track input length

outputs = model.generate(
    **inputs,
    max_new_tokens=512,
    temperature=0.7,
    do_sample=True
)

Setting `pad_token_id` to `eos_token_id`:32021 for open-end generation.


In [12]:
# Extract ONLY the new tokens (skip the prompt)
response = tokenizer.decode(outputs[0][input_length:], skip_special_tokens=True)
print(response)  # Only the model's response, not your prompt

SUMMARY: The function `handle_api_response` has been modified to validate the response data and raise an error if it contains non-dictionary elements.

ISSUES: [None]

IMPROVEMENTS: [None]

DECISION: Yes



In [13]:
response = tokenizer.decode(outputs[0], skip_special_tokens=True)

In [18]:
for data in dataset[:10]:
    prompt = prompt_generator_service.generate_style_review_prompt(
        added_code=data['added_code'],
        deleted_code=data['deleted_code'],
        full_function_code=data['full_function_code'],
        function_name=data['function_name']
    )
    messages = [
        {"role": "user", "content": prompt}
    ]
    input_text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    input_length = inputs.input_ids.shape[1]  # Track input length

    outputs = model.generate(
        **inputs,
        max_new_tokens=512,
        temperature=0.7,
        do_sample=True,
        pad_token_id=tokenizer.pad_token_id
    )
    response = tokenizer.decode(outputs[0][input_length:], skip_special_tokens=True)
    print(response)  # Only the model's response, not your prompt
    print("--------------------------------")


SUMMARY: Added code to validate response data

ISSUES: None

IMPROVEMENTS: None

DECISION: Yes

--------------------------------
SUMMARY: Added validation for response data in `handle_api_response`.

ISSUES: None found.

IMPROVEMENTS: None needed.

DECISION: No

--------------------------------
SUMMARY: Added validation for response data type in `handle_api_response` function

ISSUES: None found

IMPROVEMENTS: None needed

DECISION: Yes

--------------------------------
SUMMARY: Added validation for response data type and removed unnecessary return.

ISSUES: None found.

IMPROVEMENTS: None needed.

DECISION: Yes

--------------------------------
SUMMARY: The function now handles more than one level of API responses.

ISSUES: The function does not handle responses where the response data is not a list or dictionary.

IMPROVEMENTS: The function should handle responses where the response data is not a list or dictionary.

DECISION: Yes

The function now handles more than one level of API 